In [1]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# API Call

In [2]:
def prediction_call(locations):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": locations[0],
        "longitude": locations[1],
        "daily": ["temperature_2m_max","temperature_2m_min","rain_sum",],
        "Timezone": "auto"
    }
    responses = openmeteo.weather_api(url, params=params)

    return responses

# Process API Call

In [3]:
def extract_process(responses):
    response = responses[0]

    daily_data = response.Daily()

    temp_min = daily_data.Variables(0).ValuesAsNumpy()
    temp_max = daily_data.Variables(1).ValuesAsNumpy()
    rain_sum = daily_data.Variables(2).ValuesAsNumpy()
    response = responses[0]

    daily_data = response.Daily()
    daily_index = pd.date_range(
            start = pd.to_datetime(daily_data.Time(), unit = "s"),
            end = pd.to_datetime(daily_data.TimeEnd(), unit = "s"),
            freq = pd.Timedelta(seconds = daily_data.Interval()),
            inclusive = "left"
            )
    temp_min = daily_data.Variables(0).ValuesAsNumpy()
    temp_max = daily_data.Variables(1).ValuesAsNumpy()
    rain_sum = daily_data.Variables(2).ValuesAsNumpy()
    daily_df = {
            "Datetime" : daily_index,
            "Minimum Temperature" : temp_min,
            "Maximum Temperature" : temp_max,
            "Rain" : rain_sum
        }

    daily_df = pd.DataFrame(daily_df)
    return daily_df

    
    

In [3]:
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
locations = [52,64]
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": locations[0],
    "longitude": locations[1],
    "daily": ["temperature_2m_max","temperature_2m_min","rain_sum",],
    "Timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

daily_data = response.Daily()
daily_index = pd.date_range(
        start = pd.to_datetime(daily_data.Time(), unit = "s"),
        end = pd.to_datetime(daily_data.TimeEnd(), unit = "s"),
        freq = pd.Timedelta(seconds = daily_data.Interval()),
        inclusive = "left"
        )
temp_min = daily_data.Variables(0).ValuesAsNumpy()
temp_max = daily_data.Variables(1).ValuesAsNumpy()
rain_sum = daily_data.Variables(2).ValuesAsNumpy()
daily_df = {
        "Datetime" : daily_index,
        "Minimum Temperature" : temp_min,
        "Maximum Temperature" : temp_max,
        "Rain" : rain_sum
    }

daily_df = pd.DataFrame(daily_df)
daily_df


,Datetime,Minimum Temperature,Maximum Temperature,Rain
0,2024-08-20,22.874001,13.524,0.0
1,2024-08-21,24.374001,13.074,4.8
2,2024-08-22,18.974001,14.174,0.2
3,2024-08-23,17.174002,11.824,1.2
4,2024-08-24,18.924002,10.474,0.0
5,2024-08-25,19.274000,10.224,0.0
6,2024-08-26,20.474001,10.074,0.0
